In [16]:
import sys
home = '/home/ec2-user/git/EnsembleCpG/'
sys.path.append(home+'code/')
import pandas as pd
import numpy as np
from common import commons 
from log import Logger

In [2]:
def cal_beta(beta_file,pos_file):
    betas = pd.read_csv(beta_file,sep='\s+',index_col=['TargetID'])
    mean_betas = pd.DataFrame(betas.mean(axis=1),columns=['beta'])
    mean_betas.index = betas.index
    pos = pd.read_csv(pos_file,sep='\s+',usecols=[0,2,3],index_col=0, header=None,skiprows=1,names=['id','chr','coordinate'])
    beta_pos = mean_betas.join(pos)
    return beta_pos

In [3]:

log_dir = home+'logs/'
logger = Logger.Logger(log_dir).get_logger()
beta_file = home+'data/AD_CpG/ROSMAP_arrayMethylation_imputed.tsv'
pos_file = home+'data/AD_CpG/ROSMAP_arrayMethylation_metaData.tsv'
all_sites_betas = cal_beta(beta_file,pos_file)

In [4]:
all_sites_betas

,beta,chr,coordinate
TargetID,,,
cg00000165,0.154822,1,91194674
cg00000363,0.131544,1,230560793
cg00000957,0.779174,1,5937253
cg00001349,0.893716,1,166958439
cg00001364,0.727746,1,214170376
cg00001446,0.843326,1,43831041
cg00001534,0.882839,1,51034865
cg00001583,0.073302,1,200011786
cg00001593,0.927707,1,170490434


In [5]:
pos_sites_file = home+'data/AD_CpG/PosCpG_diff.csv'
pos_sites = pd.read_csv(pos_sites_file,usecols=[0,1,2,7,8],header=None,index_col=0,names=['id','chr','coordinate','label','pvalue'])
pos_sites['label'] = np.where(pos_sites['label']>0,1,-1)
pos_sites_betas = pos_sites.join(all_sites_betas['beta'])
neg_sites = all_sites_betas.drop(pos_sites_betas.index)
pos_sites_betas.reset_index(inplace=True)
neg_sites = neg_sites.reset_index().rename({'TargetID': 'id'})
neg_sites['label'] = 0
neg_sites.sort_values(['beta'],inplace=True)
sample_ratio_neg_to_pos = 20

In [6]:
select_negs_list = []
for beta in pos_sites_betas['beta'].values:
    neg_ix = neg_sites['beta'].searchsorted(beta)[0]
    neg_ix_lower = np.maximum(neg_ix-int(sample_ratio_neg_to_pos/2),0)
    lower_missing = int(sample_ratio_neg_to_pos/2)-(neg_ix-neg_ix_lower)
    neg_ix_upper = np.minimum(neg_ix+int(sample_ratio_neg_to_pos/2)+lower_missing,len(neg_sites))
    negs = neg_sites.iloc[neg_ix_lower:neg_ix_upper+1,:]    
    select_negs_list.extend(negs.values)
select_negs = pd.DataFrame(select_negs_list,columns=['id','beta','chr','coordinate','label'])
pos_hyper_sites = pos_sites_betas[pos_sites_betas['label']>0]
pos_hypo_sites = pos_sites_betas.query('label<0')
pos_sites_betas['beta_sign'] = pos_sites_betas['label']

In [7]:
pos_sites_betas

,id,chr,coordinate,label,pvalue,beta,beta_sign
0,cg11724984,12,121890864,1,1.270000e-07,0.775585,1
1,cg23968456,10,73521631,1,1.450000e-05,0.968990,1
2,cg15821544,1,43473840,1,1.460000e-07,0.878496,1
3,cg16733298,16,19127132,1,9.760000e-06,0.788969,1
4,cg22962123,7,27153605,1,5.560000e-08,0.264542,1
5,cg13076843,17,74475294,1,5.810000e-07,0.884712,1
6,cg25594100,7,4786943,1,1.830000e-08,0.780079,1
7,cg00621289,21,47855916,1,3.980000e-07,0.836445,1
8,cg19803550,17,1637391,1,6.810000e-07,0.966216,1
9,cg03169557,16,89598950,1,1.670000e-06,0.968596,1


In [17]:

##join with feature windows
win_path = home+'wins.txt'
pos_sites_with_winid, neg_sites_with_winid = commons.merge_with_feature_windows(win_path,pos_sites_betas,select_negs)
all_sites_with_winid = pos_sites_with_winid.append(neg_sites_with_winid,ignore_index=True)
all_sites_with_winid.drop_duplicates(['id'],inplace=True)
all_sites_with_winid.sort_values(['chr','coordinate'],inplace=True)
columns = ['id', 'chr', 'coordinate', 'beta_sign', 'pvalue', 'beta', 'label', 'start', 'winid', 'end']
all_sites_with_winid=all_sites_with_winid.reindex(columns=columns)

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/pandas/core/generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ec2-user/git/EnsembleCpG/code/common/commons.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].astype('i8')
/home/ec2-user/git/EnsembleCpG/code/common/commons.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  feature_wins.sor

In [18]:
#export all features to file
with pd.HDFStore(home+'data/AD_CpG/all_sites_winid','w') as h5s:
    h5s['all_sites_winid'] = all_sites_with_winid       
all_sites_with_winid.to_csv(home+'data/AD_CpG/all_sites_winid.csv',index=False)  
all_sites_with_winid['winid'].to_csv(home+'data/AD_CpG/selected_pos_winid.csv',index=False)

In [19]:
all_sites_with_winid

,id,chr,coordinate,beta_sign,pvalue,beta,label,start,winid,end
71,cg09756115,1,879415,NaN,NaN,0.740752,0,879401,4398,879600
72,cg22381068,1,906363,NaN,NaN,0.241956,0,906201,4532,906400
73,cg11211792,1,949634,NaN,NaN,0.779614,0,949601,4749,949800
74,cg04788999,1,949850,NaN,NaN,0.692312,0,949801,4750,950000
75,cg08530610,1,991567,NaN,NaN,0.692247,0,991401,4958,991600
76,cg14144728,1,1101462,NaN,NaN,0.788976,0,1101401,5508,1101600
77,cg13362546,1,1102960,NaN,NaN,0.780087,0,1102801,5515,1103000
78,cg07146103,1,1170069,NaN,NaN,0.798505,0,1170001,5851,1170200
79,cg01937247,1,1229534,NaN,NaN,0.761588,0,1229401,6148,1229600
80,cg03116557,1,1375480,NaN,NaN,0.878502,0,1375401,6878,1375600


In [20]:
positives = commons.sample_weights(all_sites_with_winid,all_sites_with_winid['label'],factor=1.5)

In [21]:
positives.max()

129.56869735429188